In [ ]:

# WEB SCRAPING "JobStreet" Daerah Yogyakarta Kategori Teknologi Informasi & Komunikasi

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=options)
driver.get("https://id.jobstreet.com/id/jobs-in-information-communication-technology/in-DI-Yogyakarta")

soup = BeautifulSoup(driver.page_source, "html.parser")
# print(soup)

cards = soup.select("article")

data = []

for card in cards:
    loker_tag = card.select_one("h3")
    pt_tag = card.select_one("a[data-automation='jobCompany']")
    daerah_tag = card.select_one("a[data-automation='jobLocation']")
    gaji_tag = card.select_one("span[data-automation='jobSalary']")

    loker = loker_tag if loker_tag else "Tidak Di Tampilkan"
    pt = pt_tag if pt_tag else None
    daerah = daerah_tag if daerah_tag else "Tidak Di Tampilkan"
    gaji = gaji_tag if gaji_tag else "Tidak Di Tampilkan"
    
    data.append([loker, pt, daerah, gaji])
    
df = pd.DataFrame(data, columns=["Nama Lowongan","Perusahaan","Daerah","Gaji"])
print(df)


                                        Nama Lowongan                                         Perusahaan         Domisili                                        Gaji
0                              [[[Network Engineer]]]                   [PT. Mega Bintang Mas Indonesia]     [Yogyakarta]                          Tidak Di Tampilkan
1                               [[[System Engineer]]]                    [PT Exabytes Network Indonesia]     [Yogyakarta]                          Tidak Di Tampilkan
2                      [[[Quality Assurance Tester]]]                  [PT Lightmedia Inovasi Teknologi]  [DI Yogyakarta]                          Tidak Di Tampilkan
3   [[[Admin MIS (Systems Management/ Database Adm...  [PT. Infomedia Nusantara (Telkom Group) Site J...  [DI Yogyakarta]   [[Rp 3.709.355 – Rp 4.108.600 per month]]
4                              [[[IT SPV (English)]]]                          [PT Indotel Maju Bersama]     [Yogyakarta]  [[Rp 9.000.000 – Rp 10.000.000 per month]]
5   

In [ ]:

# Cleansing Data
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/joobsteet_yogya_db")

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=options)
driver.get("https://id.jobstreet.com/id/jobs-in-information-communication-technology/in-DI-Yogyakarta")

soup = BeautifulSoup(driver.page_source, "html.parser")
# print(soup)

cards = soup.select("article")

data = []

for card in cards:
    loker_tag = card.select_one("h3")
    pt_tag = card.select_one("a[data-automation='jobCompany']")
    daerah_tag = card.select_one("a[data-automation='jobLocation']")
    gaji_tag = card.select_one("span[data-automation='jobSalary']")

    loker = loker_tag.get_text(strip=True) if loker_tag else "Tidak Di Tampilkan"
    pt = pt_tag.get_text(strip=True) if pt_tag else None
    daerah = daerah_tag.get_text(strip=True) if daerah_tag else "Tidak Di Tampilkan"
    gaji = gaji_tag.get_text(strip=True) if gaji_tag else "Tidak Di Tampilkan"
    
    data.append([loker, pt, daerah, gaji])
    
df = pd.DataFrame(data, columns=["nama_lowongan","Perusahaan","Daerah","Gaji"])

# Menghilangkan Text Yang Tidak Perlu
df["nama_lowongan"] = df["nama_lowongan"].str.replace("/", "", regex=False)
df["nama_lowongan"] = df["nama_lowongan"].str.replace("(", "", regex=False)
df["nama_lowongan"] = df["nama_lowongan"].str.replace(")", "", regex=False)

df["Perusahaan"] = df["Perusahaan"].str.replace("(", "", regex=False)
df["Perusahaan"] = df["Perusahaan"].str.replace(")", "", regex=False)

# Menghapus Lowongan Kerja Dengan Gaji Yang Tidak Di Tampilkan
# Dan Menghilangkan Text Yang Tidak Perlu
df = df[df['Gaji'] != "Tidak Di Tampilkan"]  
df['Gaji'] = df['Gaji'].str.replace('Rp', '', regex=False)
df['Gaji'] = df['Gaji'].str.replace('per month', '', regex=False)
df['Gaji'] = df['Gaji'].str.replace('/', '', regex=False)
df['Gaji'] = df['Gaji'].str.replace('–', '-')
df['Gaji'] = df['Gaji'].str.replace(',', '.') 
df['Gaji'] = df['Gaji'].str.strip()

df = df.dropna(subset=['Gaji'])
# Export ke Excell
df.to_excel("lowongan.xlsx", index=False)
# Export To SQL
df.to_sql('lowongan', con=engine, if_exists='append', index=False)

print(df)



                                        nama_lowongan                                         Perusahaan         Daerah                     Gaji
3                                      IT SPV English                            PT Indotel Maju Bersama     Yogyakarta  9.000.000 -  10.000.000
7   Admin MIS Systems Management Database Administ...  PT. Infomedia Nusantara Telkom Group Site Jogj...  DI Yogyakarta   3.709.355 -  4.108.600
10  Tech Lead  Senior Software Developer PythonDjango                  PT Mindo Small Business Solutions     Yogyakarta   5.000.000 -  7.000.000
15                     Tech Admin and Privacy Support                                PT. Dias Naga Alpha         Sleman   3.000.000 -  3.500.000
17                              Automation Programmer                  PT Mindo Small Business Solutions     Yogyakarta   4.000.000 -  6.000.000
18                                         Programmer                           PT Marel Sukses Pratamaa         Sleman   2.400.00